In [31]:
!. ./.venv/bin/activate
# 3 лаба - интегрирование (лев пр, центр пр, трапеция, Симпсон) + показать, что погрешность падает и построить интерп. многочлен 

### Lab 1
Algos to program:
1. Jacobi iterative method
2. Seidel method
3. Tridiagonal matrix algo (Thomas algorithm)
4. Any variational method (наиск. спуска, минимальной невязки)

### Трёхдиагональная прогонка

In [2]:
import numpy as np

x = np.array( [1., 2, 3, 4, 5, 6, 7] )
A = np.array([ [1., 2, 0, 0, 0, 0, 0],
               [3, 4, 5, 0, 0, 0, 0],
              [0, 6, 7, 8, 0, 0, 0],
              [0, 0, 9, 10, 11, 0, 0],
              [0, 0, 0, 12, 13, 14, 0],
              [0, 0, 0, 0, 15, 16, 17],
              [0, 0, 0, 0, 0, 17, 18] ])
f = np.dot(A, x)
print(f, A, sep='\n')

### Прогоночные коэффиценты
n = len(x)
# a
a = [0]
[ a.append(A[row][row-1]) for row in range(1, n)]
# b
b = [A[row][row] for row in range(0, n)]
# c
c = [A[row][row+1] for row in range(0, n-1)]
c.append(0)

E_0 = -c[0]/b[0]
F_0 = f[0]*b[0]

E = np.array( [0.]*n )
F = np.array( [0.]*n )
E[0] = E_0
F[0] = F_0

for k in range(0, n):
  E[k] = -c[k]/(a[k]*E[k-1]+b[k])

for k in range(0, n):
  F[k] = (f[k] - a[k]*F[k-1])/(a[k]*E[k-1]+b[k])

# Расчёт ответа
y = np.array( [0.]*n )
y[n-1] = F[n-1]
for k in range(n-2, -1 ,-1):
  y[k] = F[k] + E[k]*y[k+1]  

y

[  5.  26.  65. 122. 197. 290. 228.]
[[ 1.  2.  0.  0.  0.  0.  0.]
 [ 3.  4.  5.  0.  0.  0.  0.]
 [ 0.  6.  7.  8.  0.  0.  0.]
 [ 0.  0.  9. 10. 11.  0.  0.]
 [ 0.  0.  0. 12. 13. 14.  0.]
 [ 0.  0.  0.  0. 15. 16. 17.]
 [ 0.  0.  0.  0.  0. 17. 18.]]


array([1., 2., 3., 4., 5., 6., 7.])

### Метод Якоби

In [3]:
import numpy as np
A = np.array( [
  [10, 2, 2, 2],
  [0, 5, 0, 0],
  [1, 1, 15, 2],
  [3, 3, 3, 13]
] )
x = [1, 2, 3, 4]
f = A @ x
print(A)
print(f)

def Jacobi(A, f, x_0, eps=1e-3, max_iter=1000):
  x_cur = x_0.copy()
  diag = A.diagonal()
  for iter in range(0, max_iter):
    delta = (f - A@x_cur)/diag
    x_next = x_cur + delta
    if (np.linalg.norm(delta) < eps):
      return x_cur
    x_cur = x_next

y = Jacobi(A, f, [1,1,1,1])
y

[[10  2  2  2]
 [ 0  5  0  0]
 [ 1  1 15  2]
 [ 3  3  3 13]]
[28 10 56 70]


array([1.00016213, 2.        , 3.0001008 , 4.00017392])

### Метод Зейделя

In [4]:
from itertools import product
import numpy as np

A = np.array( [
  [10., 2, 2, 2],
  [0, 5, 0, 0],
  [1, 1, 15, 2],
  [3, 3, 3, 13]
] )
x = np.array( [1., 1, 1, 1] )
f = A@x
print(A)
print(f)

y0 = [0.4, 0.5, 4, 1]

def Seidel(A, f, y0, eps=1e-3, max_iter=1000):
  n = len(y0)

  x = np.array(y0)
  for k in range(0, max_iter):
    x_cur = x.copy()
    for j in range(0, n):
      l = 0
      for i in range(0, j):
        l += A[j][i]/A[j][j]*x[i]
      u = 0
      for i in range(j+1, n):
        u += A[j][i]/A[j][j]*x[i]
      x[j] = f[j]/A[j][j] - l - u
    if (np.linalg.norm(x_cur - x) < eps):
      return x
    
Seidel(A, f, y0, eps=1e-6, max_iter=1000)

def SimpleIter(A, f, y0, eps=1e-3, max_iter=1000):
  D = A.diagonal()
  n = len(y0)
  U = np.array( [ (A[i][j] if i < j else 0) for i, j in product(range(0, n), repeat=2) ] )
  U = U.reshape(n,n)

  L = np.array( [ (A[i][j] if i > j else 0) for i, j in product(range(0, n), repeat=2) ] )
  L = L.reshape(n,n)

  y_cur = y0.copy()
  for k in range(0, max_iter):
    y_next = f/D - ((L + U)@y_cur)/D
    if( np.linalg.norm(y_next - y_cur) < eps ):
      return y_next
    y_cur = y_next.copy()
                 
SimpleIter(A, f, y0)


[[10.  2.  2.  2.]
 [ 0.  5.  0.  0.]
 [ 1.  1. 15.  2.]
 [ 3.  3.  3. 13.]]
[16.  5. 19. 22.]


array([0.99994462, 1.        , 0.99996565, 0.99993984])

### Вариационный метод (метод наиск. спуска)

In [5]:
import numpy as np

A = np.array( [
  [10., 2, 2, 2],
  [0, 5, 0, 0],
  [1, 1, 15, 2],
  [3, 3, 3, 13]
] )
#A = A.T.dot(A)
x = np.array( [3., 100, 5, 19] )
f = A@x
print(A)
print(f)

y0 = [0.4, 0.5, 4, 1]

print(np.linalg.eigvals(A))

def fast_descend(A, f, y0, eps=1e-6, max_iter=1000):
  y_cur = y0.copy()

  for k in range(max_iter):
    r = A@y_cur - f
    t = np.dot(r, r)/(np.dot(A@r, r))
    y_next = y_cur - t*(A@y_cur - f)
    if( np.linalg.norm(y_next - y_cur) < eps ):
      return y_next
    y_cur = y_next.copy()

fast_descend(A, f, y0)

[[10.  2.  2.  2.]
 [ 0.  5.  0.  0.]
 [ 1.  1. 15.  2.]
 [ 3.  3.  3. 13.]]
[278. 500. 216. 571.]
[17.55197369  8.66399239 11.78403391  5.        ]


array([ 3.00000029, 99.99999919,  5.00000011, 19.00000031])